<a href="https://colab.research.google.com/github/liuyixi520/code/blob/main/baseline_gd_bert_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 数据

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
train_df = pd.read_excel("./drive/MyDrive/good_doctor/data_train.xlsx")
test_df = pd.read_excel("./drive/MyDrive/good_doctor/data_test.xlsx")
test_submit = pd.read_csv("./drive/MyDrive/good_doctor/baseline.csv")

In [ ]:
!pip install transformers torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 22.9 MB/s 
     |████████████████████████████████| 86 kB 7.0 MB/s 
     |████████████████████████████████| 596 kB 64.7 MB/s 
     |████████████████████████████████| 6.6 MB 58.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from torch import nn
from transformers import BertPreTrainedModel
from transformers import AutoTokenizer 
from transformers import AutoModelForMaskedLM 
from transformers import AutoConfig
from transformers import BertModel
from transformers import AutoModel

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-roberta-wwm-ext")
config = AutoConfig.from_pretrained("hfl/chinese-roberta-wwm-ext")

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
class XunFeiModel(nn.Module):
    def __init__(self, num_labels_i, num_labels_j): 
        super(XunFeiModel,self).__init__() 

        #Load Model with given checkpoint and extract its body
        self.model = model = AutoModel.from_pretrained("hfl/chinese-roberta-wwm-ext")
        self.dropout = nn.Dropout(0.1) 
        self.classifier_i = nn.Linear(768, num_labels_i)
        self.classifier_j = nn.Linear(768, num_labels_j)

    def forward(self, input_ids=None, attention_mask=None,labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = self.dropout(outputs[0]) #outputs[0]=last hidden state
    
        logits_i = self.classifier_i(sequence_output[:,0,:].view(-1,768))
        logits_j = self.classifier_j(sequence_output[:,0,:].view(-1,768))
        
        return logits_i, logits_j

In [ ]:
train_text = train_df['diseaseName'] + ' ' + train_df['conditionDesc'] + ' ' + train_df['title'] + ' ' + train_df['hopeHelp']
test_text = test_df['diseaseName'] + ' ' + test_df['conditionDesc'] + ' ' + test_df['title'] + ' ' + test_df['hopeHelp']
train_text = train_text.fillna('')
test_text = test_text.fillna('')

train_encoding = tokenizer(train_text.tolist()[:-1000], truncation=True, padding=True, max_length=200)
val_encoding = tokenizer(train_text.tolist()[-1000:], truncation=True, padding=True, max_length=200)
test_encoding = tokenizer(test_text.tolist(), truncation=True, padding=True, max_length=200)



In [ ]:
from torch.utils.data import Dataset, DataLoader, TensorDataset

# 数据集读取
class XunFeiDataset(Dataset):
    def __init__(self, encodings, label_i, label_j):
        self.encodings = encodings
        self.label_i = label_i
        self.label_j = label_j
    
    # 读取单个样本
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label_i'] = torch.tensor(int(self.label_i[idx]))
        item['label_j'] = torch.tensor(int(self.label_j[idx]))
        return item
    
    def __len__(self):
        return len(self.label_i)

train_dataset = XunFeiDataset(train_encoding, 
                              train_df['label_i'].values[:-1000], 
                              train_df['label_j'].values[:-1000])
val_dataset = XunFeiDataset(val_encoding, 
                              train_df['label_i'].values[-1000:], 
                              train_df['label_j'].values[-1000:])
test_dataset = XunFeiDataset(test_encoding, [0] * len(test_df), [0] * len(test_df))


In [ ]:
# 单个读取到批量读取
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
model = XunFeiModel(20, 61)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda


In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW

loss_fn = CrossEntropyLoss()
optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
def train():
    model.train()
    total_train_loss = 0
    iter_num = 0
    total_iter = len(train_loader)
    for batch in train_loader:
        # 正向传播
        optim.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        label_i = batch['label_i'].to(device)
        label_j = batch['label_j'].to(device)

        pred_i, pred_j = model(
            input_ids, 
            attention_mask
        )
        
        valid = label_j != -1
        loss = loss_fn(pred_i, label_i)  + loss_fn(pred_j[valid], label_j[valid])
        # 反向梯度信息
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # 参数更新
        optim.step()

        iter_num += 1
        
        if(iter_num % 100 == 0):
            print("iter_num: %d, loss: %.4f, %.2f%% %.4f / %.4f" % (
                iter_num, loss.item(), iter_num/total_iter*100, 
                (pred_i.argmax(1) == label_i).float().data.cpu().numpy().mean(),
                (pred_j[valid].argmax(1) == label_j[valid]).float().data.cpu().numpy().mean()
            ))

def validation():
    model.eval()
    label_i_acc, label_j_acc = 0, 0
    for batch in val_dataloader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            label_i = batch['label_i'].to(device)
            label_j = batch['label_j'].to(device)

            pred_i, pred_j = model(
                input_ids, 
                attention_mask
            )
    
            valid = label_j != -1
            label_i_acc += (pred_i.argmax(1) == label_i).float().sum().item()
            label_j_acc += (pred_j[valid].argmax(1) == label_j[valid]).float().sum().item()
    
    label_i_acc = label_i_acc / len(val_dataloader.dataset)
    label_j_acc = label_j_acc / len(val_dataloader.dataset)

    print("-------------------------------")
    print("Accuracy: %.4f / %.4f" % (label_i_acc, label_j_acc))
    print("-------------------------------")

In [ ]:
def prediction():
    model.eval()
    test_label_i = []
    test_label_j = []
    for batch in test_dataloader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            label_i = batch['label_i'].to(device)
            label_j = batch['label_j'].to(device)

            pred_i, pred_j = model(input_ids, attention_mask)
            test_label_i += list(pred_i.argmax(1).data.cpu().numpy())
            test_label_j += list(pred_j.argmax(1).data.cpu().numpy())
    return test_label_i, test_label_j

In [ ]:
for epoch in range(2):
    train()
    validation()
    
test_pred_i, test_pred_j = prediction()

iter_num: 100, loss: 3.6625, 7.32% 0.7500 / 0.3000
iter_num: 200, loss: 2.0819, 14.63% 0.7500 / 0.6250
iter_num: 300, loss: 1.2008, 21.95% 0.9375 / 0.9000
iter_num: 400, loss: 1.5238, 29.26% 0.9375 / 0.7273
iter_num: 500, loss: 4.6304, 36.58% 0.6250 / 0.1429
iter_num: 600, loss: 1.7698, 43.89% 0.8125 / 0.7273
iter_num: 700, loss: 1.3737, 51.21% 1.0000 / 0.5000
iter_num: 800, loss: 2.3154, 58.52% 0.8750 / 0.6000
iter_num: 900, loss: 1.8914, 65.84% 0.7500 / 0.5714
iter_num: 1000, loss: 0.6908, 73.15% 0.8125 / 1.0000
iter_num: 1100, loss: 2.1290, 80.47% 0.6875 / 0.3750
iter_num: 1200, loss: 1.6145, 87.78% 0.8125 / 0.7143
iter_num: 1300, loss: 1.5662, 95.10% 0.7500 / 0.6000
-------------------------------
Accuracy: 0.8230 / 0.4040
-------------------------------
iter_num: 100, loss: 0.9125, 7.32% 0.9375 / 0.8000
iter_num: 200, loss: 1.6735, 14.63% 0.8125 / 0.7500
iter_num: 300, loss: 0.3094, 21.95% 1.0000 / 1.0000
iter_num: 400, loss: 0.3229, 29.26% 1.0000 / 1.0000
iter_num: 500, loss: 2.8

In [ ]:
test_pred_i, test_pred_j = prediction()

test_submit['label_i'] = test_pred_i
test_submit['label_j'] = test_pred_j

test_submit.to_csv('./drive/MyDrive/good_doctor/bert_submit.csv', index=None)